In [7]:
%run Common.ipynb

C:\Source\CSIRO\HealthyHabitatAI\.env
healthyhabitatsvb
0crB6HIwJxefIX/Op19ZVLepgE41jCp/SYTnlBGpPBB0To9Jrpf37pLvWX/eMhNyIVM8JxBC+n94IQOH1Cf9/g==
processed
190ee425b5d9441f916dfbc6246fc631
bf6bff18-3b51-4b2d-ba40-78b4d2e9d7cf
b8b9b6ca-5dd5-42ac-a2f1-70dd5dd24de9


In [8]:
import numpy as np
import matplotlib.pyplot as plt

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry
from PIL import Image

In [9]:
%matplotlib inline

In [10]:
Image.MAX_IMAGE_PIXELS = None

### Custom Vision

In [11]:
ENDPOINT = 'https://australiaeast.api.cognitive.microsoft.com/'

trainer = CustomVisionTrainingClient(custom_vision_training_key, endpoint=ENDPOINT)

projects = trainer.get_projects()

In [12]:
for project in projects:
    print(project.id)
    print(project.name)

b8b9b6ca-5dd5-42ac-a2f1-70dd5dd24de9
SvBObjectDetectionDemo
bf6bff18-3b51-4b2d-ba40-78b4d2e9d7cf
SvBClassificationDemo
d486111a-2528-4e20-995d-e393ee136937
KakaduFishAI
92f89ec8-2869-45f3-9735-a423dfbd9de5
WildPigAI
d87ec242-1fc1-4f1a-9980-6833a06c8f4e
PrawnAI
60af9410-78b3-4ac5-bf5a-4995df108df5
ViaToCustomVision.ai
671a7fcf-09fd-4630-b17f-f3f9fba01424
cannonhill-wurrkeng-animals
8e325309-2e18-487c-903a-747936fe6b73
cannonhill-wurrkeng-parragrass
a98d1349-1493-4ace-86eb-38f33105d5fc
ubir-kunumeleng-animals
7fd39d75-f8eb-4778-b0c0-37a08b551e99
ubir-wurrkeng-animals
ee5572be-f570-4574-81ec-ea5840d208c4
ubir-wurrkeng-parragrass
39ce5e4d-67d3-4fc3-9e61-2a5146a3d8f4
ubir-kunumeleng-parragrass
a5468b90-f94c-4ae9-ad4e-67a6fd7a8dbb
ubir-bangkerreng-animals
187f3abf-b4a6-47a3-b126-a53e14728bbc
ubir-bangkerreng-parragrass
8ea774dc-23d5-4f9a-8202-a1f113458634
ubir-kurrung-animals
3a4ca06f-a406-44a7-8079-cdf1a1ca3ec6
ubir-kurrung-parragrass
2bdf0488-e15e-4b88-b5ff-c6c851dbfeb9
ubir-kudjewk-animal

### Blob Storage

In [13]:
containers = blob_service_list_containers(healthy_habitat_ai_storage_account_name, healthy_habitat_ai_storage_account_key)

for container_name in containers:
    print(container_name)

azure-webjobs-hosts
azure-webjobs-secrets
cannonhill-bangkerreng
cannonhill-kunumeleng
cannonhill-wurrkeng
jabirudreaming-bangkerreng
jabirudreaming-kunumeleng
jabirudreaming-wurrkeng
raw
resized
scm-releases
site-season
ubir-bangkerreng
ubir-kunumeleng
ubir-wurrkeng


### Create regions

In [14]:
for container_name in containers:
    if container_name == 'site-season':
        blobs = blob_service_list_blobs(healthy_habitat_ai_storage_account_name, healthy_habitat_ai_storage_account_key, container_name)

        folders = set([blob.split('/')[0] for blob in blobs])

        for folder in folders:
            interim_folder_path = os.path.join(Path(os.getcwd()).parent, 'data', 'interim', container_name, folder)
            if not os.path.exists(interim_folder_path):
                os.makedirs(interim_folder_path)

        for blob_name in blobs:
            folder = blob_name.split('/')[0]
            file = blob_name.split('/')[1]
            
            filter = "PartitionKey eq '{0}' and RowKey eq '{1}'".format(folder, file)
            
            entities = table_service_query_entities(healthy_habitat_ai_storage_account_name, healthy_habitat_ai_storage_account_key, healthy_habitat_ai_processed_table_name, filter)
            
            count = len([entity for entity in entities])

            if count == 0:
                print('Processing {0}'.format(blob_name))
                file_path = os.path.join(interim_folder_path, file)

                if not os.path.exists(file_path):
                    blob_service_get_blob_to_path(healthy_habitat_ai_storage_account_name, healthy_habitat_ai_storage_account_key, container_name, blob_name, file_path)

                image = np.array(Image.open(file_path))
                image_shape = image.shape

                height = 228
                width = 304

                count = 0

                for y in range(0, image_shape[0], height):
                    for x in range(0, image_shape[1], width):
                        region = image[y:y + height, x:x + width]

                        file_name = '{0}_Region_{1}.jpg'.format(file.split('.')[0], count)

                        buffer = io.BytesIO()
                        Image.fromarray(region).save(buffer, format='JPEG')

                        images = []

                        images.append(ImageFileCreateEntry(name=file_name, contents=buffer.getvalue()))

                        print('Uploading to Classification Project...')
                        result = trainer.create_images_from_files(custom_vision_para_grass_project_id, images=images)

                        if not result.is_batch_successful:
                            for image_create_result in result.images:
                                print("Image status: ", image_create_result.status)
                                
                        print('Uploading to Object Detection Project...')
                        result = trainer.create_images_from_files(custom_vision_magpie_geese_project_id, images=images)

                        if not result.is_batch_successful:
                            for image_create_result in result.images:
                                print("Image status: ", image_create_result.status)

                        count += 1

                entity = {'PartitionKey': folder, 'RowKey': file}
                table_service_insert(healthy_habitat_ai_storage_account_name, healthy_habitat_ai_storage_account_key, healthy_habitat_ai_processed_table_name, entity)
                
                os.remove(file_path)
                print(count)

Processing 2019-11-05-1100/Orthomosaic.geo.tif


OSError: cannot identify image file 'C:\\Source\\CSIRO\\HealthyHabitatAI\\data\\interim\\site-season\\2019-11-05-1100\\Orthomosaic.geo.tif'

In [6]:
from PIL import TiffImagePlugin
TiffImagePlugin.DEBUG = True
with open('C:\\Source\\CSIRO\\HealthyHabitatAI\\data\\interim\\site-season\\2019-11-05-1100\\Orthomosaic.geo.tif', 'rb') as f:
    TiffImagePlugin.TiffImageFile(f)

SyntaxError: not a TIFF file (header b'II+\x00\x08\x00\x00\x00' not valid) (<string>)

In [2]:
import tifffile as tiff
a = tiff.imread('C:\\Users\\stvanbod\\Downloads\\Orthomosaic.geo.tif')

In [5]:
image = np.array(a)
image.shape

(36097, 24869, 4)

In [10]:
import gdal

ModuleNotFoundError: No module named 'gdal'

In [ ]:
import sys
!{sys.executable} -m pip install gdal

In [ ]:
tagged_image_count = trainer.get_tagged_image_count(custom_vision_para_grass_project_id)
print('Tagged image count:', tagged_image_count)

untagged_image_count = trainer.get_untagged_image_count(custom_vision_para_grass_project_id)
print('Untagged image count:', untagged_image_count)

print('Image count', tagged_image_count + untagged_image_count)